In [307]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
# magic command to display matplotlib plots inline within the ipython notebook
%matplotlib inline
# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import catboost as cb
import urllib
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import warnings
import datetime
warnings.simplefilter("ignore")
from datetime import datetime
from collections import namedtuple
import pickle
from IPython.display import clear_output
from selenium import webdriver
from time import sleep
import re
import lightgbm as lgb
from sklearn import metrics

In [308]:
def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))
def update_address(address):
    if re.match(r'.+\sG\s+', address):
        return re.sub('\sG\s+', '', address)
    else:
        return address

In [309]:
#https://geocode-maps.yandex.ru/1.x/?geocode=Moscow+Lva+Tolstogo+Street+16&lang=en_US
import requests
import json
YANDEX_URL = "https://geocode-maps.yandex.ru/1.x"
STATIC = "/?format=json&lang=en_US&geocode="
def geocode(address):
        response = requests.get(YANDEX_URL + STATIC + address)
        parsed_dict = json.loads(response.content)
        try:
            pos =  parsed_dict['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']
            lat, lon = [float(item) for item in pos.split()]
            return lat, lon
        except:
            return None, None

# Чтение исходных данных

In [310]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
train['isTrain'] = True
test['isTrain'] = False

# Создадим таблицу, с которой будем работать далее

In [311]:
X = train.append(test, ignore_index=True)

In [312]:
X.head()

,address,address_rus,atm_group,id,isTrain,lat,long,target
0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",32.0,8526.0,True,46.940995,142.738319,0.011500
1,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",32.0,8532.0,True,46.937353,142.753348,0.029710
2,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",32.0,8533.0,True,46.959413,142.741113,0.009540
3,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",32.0,8684.0,True,55.805827,37.515146,-0.094035
4,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",32.0,37.0,True,69.343541,88.211228,0.079277


In [313]:
X.isnull().sum()

address           0
address_rus     420
atm_group         0
id                0
isTrain           0
lat             420
long            420
target         2504
dtype: int64

Как видно из последней ячейки, в наших данных есть много пропусков. Часть из них можно легко заполнить, например 
значения lat и long с помощью геокодера Яндекса.

Как можно было увидеть, изучая адреса на английском, часть из них записана в формате, которые не сможет прочесть геокодер Яндекса. Поэтому применим к таким адресам небольшое преобразование:

In [314]:
X.address = X.address.apply(lambda x: update_address(x))

## Заполнение пропусков с помощью геокодера Яндекса

In [315]:
X[['long','lat']] = X.apply(lambda row: geocode(row['address']) if (np.isnan(row['lat'])|np.isnan(row['long']))\
                                    else (row['long'],row['lat']), axis = 1).apply(pd.Series)

In [316]:
X.shape

(8765, 8)

In [317]:
X.isnull().sum()

address           0
address_rus     420
atm_group         0
id                0
isTrain           0
lat              76
long             76
target         2504
dtype: int64

In [318]:
X[X.lat.isnull()==True].head()

,address,address_rus,atm_group,id,isTrain,lat,long,target
40,PER. BAZARNY 4 SPASSK-DALNY,NaN,496.5,4510.0,True,NaN,NaN,0.039191
150,ABB 6B NAB-CHELNINSKIJ NAB.CHELNY,NaN,1022.0,347.0,True,NaN,NaN,-0.145001
475,"KR.G. ROSSIJSKAJA,6 OMSK",NaN,3185.5,2746.0,True,NaN,NaN,-0.122659
506,"D. 23, KORP. 1, UL. MATRO BIYSK",NaN,5478.0,8436.0,True,NaN,NaN,-0.137240
566,"D. 91, PR-KT OBVODNYI KAN ARKHANGELSK",NaN,5478.0,5021.0,True,NaN,NaN,-0.069187


76 адресов все же не удалось найти с помощью геокодера, заполнила их руками.

In [320]:
X = X[X.lat.isnull()==False]
X_old = pd.read_csv('X_without_nulls.csv')
X_2 = X.append(X_old, ignore_index = True)
X_2.drop_duplicates(['address','id'], inplace = True, keep = 'first')

In [321]:
X_2.shape

(8765, 9)

In [322]:
X_2.isnull().sum()

Unnamed: 0     8689
address           0
address_rus     420
atm_group         0
id                0
isTrain           0
lat               0
long              1
target         2504
dtype: int64

In [180]:
#row = 2259
#lat, longg = 55.682336, 52.295119
#X.at[row,'lat'] = lat
#X.at[row,'long'] = longg

In [181]:
#X.to_csv('X_for_model.csv')

In [182]:
#X = pd.read_csv('X_for_model.csv')

In [323]:
X = X_2.copy()

# Количество банкоматов стоящих по одному адресу

In [324]:
counts = X.groupby(['lat','long'])['id'].count().reset_index().rename(columns={'id':'count'})
X = pd.merge(X, counts, how='left', on=['lat','long'])
X.head()

,Unnamed: 0,address,address_rus,atm_group,id,isTrain,lat,long,target,count
0,NaN,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",32.0,8526.0,True,46.940995,142.738319,0.011500,1.0
1,NaN,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",32.0,8532.0,True,46.937353,142.753348,0.029710,1.0
2,NaN,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",32.0,8533.0,True,46.959413,142.741113,0.009540,1.0
3,NaN,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",32.0,8684.0,True,55.805827,37.515146,-0.094035,1.0
4,NaN,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",32.0,37.0,True,69.343541,88.211228,0.079277,1.0


# Город размещения

In [325]:
X['city'] = X[~X.address_rus.isnull()].address_rus.apply(lambda x: x.split(',')[2])
rare_cities = X.city.value_counts()[(X.city.value_counts() < 20) ==True].index
X.city = X.city.apply(lambda x: 'RARE' if x in rare_cities else x)
X.city= X.city.rank().fillna(-1)
X.head()

,Unnamed: 0,address,address_rus,atm_group,id,isTrain,lat,long,target,count,city
0,NaN,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",32.0,8526.0,True,46.940995,142.738319,0.011500,1.0,5272.0
1,NaN,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",32.0,8532.0,True,46.937353,142.753348,0.029710,1.0,5272.0
2,NaN,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",32.0,8533.0,True,46.959413,142.741113,0.009540,1.0,6926.5
3,NaN,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",32.0,8684.0,True,55.805827,37.515146,-0.094035,1.0,2212.5
4,NaN,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",32.0,37.0,True,69.343541,88.211228,0.079277,1.0,3152.5


# Расстояния до ближайших банкоматов

In [326]:
R = 6373.0 # радиус земли в километрах

def distance(x,y):
    """
    Параметры
    ----------
    x : tuple, широта и долгота первой геокоординаты 
    y : tuple, широта и долгота второй геокоординаты 
    
    Результат
    ----------
    result : дистанция в километрах между двумя геокоординатами
    """
    lat_a, long_a, lat_b, long_b = map(radians, [x[0],x[1],y[0],y[1]])    
    dlon = long_b - long_a
    dlat = lat_b - lat_a
    a = sin(dlat/2)**2 + cos(lat_a) * cos(lat_b) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

In [327]:
knc = KNeighborsClassifier(metric=distance)
dots = X[['lat','long']].dropna()
knc.fit(X=dots , y=np.ones(dots.shape[0]))
distances, indexes = knc.kneighbors(X=dots,n_neighbors=6,)
for i in range(1,4):
    dots['distance_%s'%i] = distances[:,i]
    dots['indexes_%s'%i] = indexes[:,i]
dots['mean'] = dots.iloc[:,dots.columns.str.contains('distance')].mean(axis=1)
dots = dots.drop(columns = ['lat','long'])
X = pd.concat([X,dots], axis=1)

# Численность населения города

In [328]:
X['city_name'] = X[~X.address_rus.isnull()].address_rus.apply(lambda x: x.split(',')[2])
cities = pd.read_excel('city2.xlsx')
cities.city_name = cities.city_name.apply(lambda x: x.encode('utf-8'))
X = pd.merge(X, cities, how='left', on='city_name')
X.population = X.population.apply(lambda x: str(x).replace(' ', '')).replace('nan', 0).astype(int)
X.head()

,Unnamed: 0,address,address_rus,atm_group,id,isTrain,lat,long,target,count,...,distance_1,indexes_1,distance_2,indexes_2,distance_3,indexes_3,mean,city_name,№,population
0,NaN,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",32.0,8526.0,True,46.940995,142.738319,0.011500,1.0,...,0.171318,8628.0,0.406053,6017.0,0.447444,3718.0,0.341605,Южно-Сахалинск,168.0,194882
1,NaN,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",32.0,8532.0,True,46.937353,142.753348,0.029710,1.0,...,0.243277,1703.0,0.649501,433.0,0.720783,8215.0,0.537854,Южно-Сахалинск,168.0,194882
2,NaN,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",32.0,8533.0,True,46.959413,142.741113,0.009540,1.0,...,0.063770,8717.0,0.221691,2491.0,0.221691,4191.0,0.169050,Сахалинская область,NaN,0
3,NaN,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",32.0,8684.0,True,55.805827,37.515146,-0.094035,1.0,...,0.121670,5171.0,0.121670,5168.0,0.121670,5169.0,0.121670,Москва,75.0,12380664
4,NaN,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",32.0,37.0,True,69.343541,88.211228,0.079277,1.0,...,0.199516,4752.0,0.367687,3555.0,0.394887,3527.0,0.320697,Норильск,100.0,178018


# Стоит ли банкомат в отделении Росбанка

In [329]:
dep = pd.read_csv('departments_with_lat.csv')
dep = dep.drop_duplicates(['lat','long'])
X = pd.merge(X, dep, how='left', on=['lat','long'])
X.is_department = X.is_department.fillna(0)
X.shape

(8765, 25)

In [330]:
len(X[X.is_department==1])

425

# Подготовка данных для модели

In [331]:
X.atm_group = X.atm_group.astype(int)
X.lat, X.long = X.lat.astype(float), X.long.astype(float)

In [332]:
X_ = X[X.isTrain][['atm_group', 'lat', 'long',  'count', 'distance_1',
                    'distance_2',  'distance_3', 'mean', 'population','id','is_department']]
Y_ = X.loc[X.isTrain, 'target']

X_test = X[~X.isTrain][['atm_group', 'lat', 'long',  'count', 'distance_1',
                    'distance_2',  'distance_3','mean', 'population','id','is_department']]

In [333]:
X_.head()

,atm_group,lat,long,count,distance_1,distance_2,distance_3,mean,population,id,is_department
0,32,46.940995,142.738319,1.0,0.171318,0.406053,0.447444,0.341605,194882,8526.0,0.0
1,32,46.937353,142.753348,1.0,0.243277,0.649501,0.720783,0.537854,194882,8532.0,0.0
2,32,46.959413,142.741113,1.0,0.063770,0.221691,0.221691,0.169050,0,8533.0,0.0
3,32,55.805827,37.515146,1.0,0.121670,0.121670,0.121670,0.121670,12380664,8684.0,0.0
4,32,69.343541,88.211228,1.0,0.199516,0.367687,0.394887,0.320697,178018,37.0,0.0


In [335]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X_, Y_, test_size=0.25, random_state=1)

# Catboost

In [337]:
categorical = ['atm_group']

In [338]:
def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols,query_cols,sorter=sidx)]
categorical_features_pos = column_index(X_train,categorical)

In [340]:
cb_model = cb.CatBoostRegressor(iterations=1000,
                             learning_rate=0.02,
                             depth=12,
                             one_hot_max_size=10,
                             eval_metric='RMSE',
                             random_seed = 23,
                             cat_features = categorical_features_pos ,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 75,
                             od_wait=100,)

In [341]:
cb_model.fit(X_train, Y_train,
             eval_set=(X_valid,Y_valid),
             use_best_model=True)

0:	learn: 0.0852179	test: 0.0835126	best: 0.0835126 (0)	total: 1.44s	remaining: 23m 56s
75:	learn: 0.0493118	test: 0.0492834	best: 0.0492834 (75)	total: 25.8s	remaining: 5m 13s
150:	learn: 0.0446653	test: 0.0454144	best: 0.0454144 (150)	total: 50.1s	remaining: 4m 41s
225:	learn: 0.0433466	test: 0.0447084	best: 0.0447084 (225)	total: 1m 12s	remaining: 4m 7s
300:	learn: 0.0425372	test: 0.0443466	best: 0.0443466 (300)	total: 1m 30s	remaining: 3m 30s
375:	learn: 0.0419217	test: 0.0441519	best: 0.0441519 (375)	total: 1m 43s	remaining: 2m 51s
450:	learn: 0.0415982	test: 0.0440318	best: 0.0440318 (450)	total: 1m 54s	remaining: 2m 19s
525:	learn: 0.0412419	test: 0.0439329	best: 0.0439329 (525)	total: 2m 5s	remaining: 1m 53s
600:	learn: 0.0408620	test: 0.0438333	best: 0.0438322 (597)	total: 2m 22s	remaining: 1m 34s
675:	learn: 0.0403764	test: 0.0437571	best: 0.0437566 (669)	total: 2m 47s	remaining: 1m 20s
750:	learn: 0.0402544	test: 0.0437176	best: 0.0437176 (729)	total: 2m 57s	remaining: 59s
8

In [342]:
#iterations 1000
rmse(Y_valid, cb_model.predict(X_valid)) 

0.04349070912927313

In [343]:
submit_cb = pd.DataFrame(cb_model.predict(X_test), index=test.index,columns=['target'])

# LightGBM

In [344]:
lgb_model = lgb.LGBMRegressor(objective = 'regression',  
                             max_depth = 10,
                             colsample_bytre = 0.8,
                             subsample = 0.8, 
                             learning_rate = 0.01,
                             n_estimators = 1000)

In [345]:
lgb_model.fit(X_train,Y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytre=0.8,
       colsample_bytree=1.0, importance_type='split', learning_rate=0.01,
       max_depth=10, min_child_samples=20, min_child_weight=0.001,
       min_split_gain=0.0, n_estimators=1000, n_jobs=-1, num_leaves=31,
       objective='regression', random_state=None, reg_alpha=0.0,
       reg_lambda=0.0, silent=True, subsample=0.8,
       subsample_for_bin=200000, subsample_freq=0)

In [346]:
rmse(Y_valid, lgb_model.predict(X_valid))

0.04392725695402974

In [347]:
submit_lgb = pd.DataFrame(lgb_model.predict(X_test), index=test.index,columns=['target'])

In [348]:
submit = 0.7*submit_cb + 0.3*submit_lgb

In [298]:
#submit_cb.to_csv('./submits/submit_22.csv') 

In [349]:
submit

,target
0,-0.041447
1,-0.010197
2,-0.018761
3,-0.042017
4,0.005042
5,-0.024377
6,-0.011264
7,-0.062654
8,-0.050591
9,-0.055561
